In [1]:
import casperfpga
import matplotlib.pyplot as plt
import time
import numpy as np

In [ ]:
#Reset the trigger counter
def reset_trigger_counter_all():
        brd9.write_int("trig_debug_reset",1)
        brd11.write_int("trig_debug_reset",1)
        brd12.write_int("trig_debug_reset",1)
        time.sleep(0.01)
        brd9.write_int("trig_debug_reset",0)
        brd11.write_int("trig_debug_reset",0)
        brd12.write_int("trig_debug_reset",0)
        return

#reset the trigger timer
def reset_trigger_timer_all():
        brd9.write_int("trig_debug_timer_reset",1)
        brd11.write_int("trig_debug_timer_reset",1)
        brd12.write_int("trig_debug_timer_reset",1)
        time.sleep(0.01)
        brd9.write_int("trig_debug_timer_reset",0)
        brd11.write_int("trig_debug_timer_reset",0)
        brd12.write_int("trig_debug_timer_reset",0)
        return

#set all boards to listen
def reset_to_listen_all():
        brd9.write_int("reset_to_listen",0)
        brd11.write_int("reset_to_listen",0)
        brd12.write_int("reset_to_listen",0)
        time.sleep(0.01)
        brd9.write_int("reset_to_listen",1)
        brd11.write_int("reset_to_listen",1)
        brd12.write_int("reset_to_listen",1)
        return
#read the readout state value on all the boards
def read_state_value_all():
        state9=brd9.read_int("readout_state_value")
        state11=brd11.read_int("readout_state_value")
        state12=brd12.read_int("readout_state_value")
        return(state9,state11,state12)

#send a trigger to one board
def trigger(board):
        #board is a casperfpga object
        board.write_int("send_trigger",0)
        time.sleep(0.01)
        board.write_int("send_trigger",1)
        return

#read the trigger counter on all the boards
def read_trigger_counter_all():
        count9=brd9.read_int("trig_debug_trig_rec_count")
        count11=brd11.read_int("trig_debug_trig_rec_count")
        count12=brd12.read_int("trig_debug_trig_rec_count")
        return count9,count11,count12


def time_trigger_communication(board):
        #board is the board to send the trigger to
        reset_trigger_counter_all()
        reset_trigger_timer_all()
        reset_to_listen_all()
        initialstate9,initialstate11,initialstate12=read_state_value_all()
        trigger(board)
        time.sleep(0.5)
        timer_value=board.read_int("trig_debug_timer")
        count9,count11,count12=read_trigger_counter_all()
        finalstate9,finalstate11,finalstate12=read_state_value_all()
        return np.asarray([initialstate9,initialstate11,initialstate12,timer_value,count9,count11,count12,finalstate9,finalstate11,finalstate12])


In [13]:
#connect to boards and load firmware

firmware = "gpio_pins3_2021-02-07_2206.fpg"

brdnames = ['snap01','snap02','snap03','snap04','snap05','snap06','snap07','snap08','snap09','snap10','snap11']
print("Connecting to boards and loading firmware")
brds = [casperfpga.CasperFpga(n) for n in brdnames]
[brd.upload_to_ram_and_program(firmware) for brd in brds]
[print(brdnames[i],'Is connected? ', brds[i].is_connected()) for i in range(len(brds))]

Connecting to boards and loading firmware


FileNotFoundError: [Errno 2] No such file or directory: 'gpio_pins3_2021-02-07_2206.fpg'

In [11]:

brd.upload_to_ram_and_program()

snap01 Is connected?  True
snap02 Is connected?  True
snap03 Is connected?  True
snap04 Is connected?  True
snap05 Is connected?  True
snap06 Is connected?  True
snap07 Is connected?  True
snap08 Is connected?  True
snap09 Is connected?  True
snap10 Is connected?  True
snap11 Is connected?  True


[None, None, None, None, None, None, None, None, None, None, None]

In [6]:
print(len(brdnames)

['snap01', 'snap02', 'snap03', 'snap04', 'snap05', 'snap06', 'snap07', 'snap08', 'snap09', 'snap10', 'snap11']
